In [ ]:
!pwd
# Load libraries
import torch
import torch.nn as nn
from torch.nn import functional as F

/content


In [ ]:
# Read in the dataset to train on
with open("drive/MyDrive/data/input.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [ ]:
print("Length of data in characters: ", len(text))

Length of data in characters:  1115394


In [ ]:
# Get unique characters
chars = sorted(list(set(text)))
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# Map characters to integers (can use tiktoken library to do this)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
# Encoder: take a string; output a list of integers
encode = lambda s: [stoi[c] for c in s]
# Decoder: take a list of integers, output a string
decode = lambda l: "".join(itos[i] for i in l)
# Test the encoder and decoder
print(encode("Hello!"))
print(decode(encode("Hello!")))

[20, 43, 50, 50, 53, 2]
Hello!


In [ ]:
# Encode the whole text dataset and store it in a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [ ]:
# Split the data into training and validation data
n = int(0.9*len(data)) # 90% for training data
train_data = data[:n]
val_data = data[n:]

In [ ]:
# Train train_data with the transformer - time dimension
# To avoid being computationally expensive, train with chunks of the dataset
block_size = 8
x = train_data[:block_size] # characters of block_size
y = train_data[1:block_size+1] # the next block_size characters
for t in range(block_size):
  context = x[:t+1] # make the transformer see from a character to block_size
  target = y[t]
  print(f"When the input is {context}, the target is: {target}")

When the input is tensor([18]), the target is: 47
When the input is tensor([18, 47]), the target is: 56
When the input is tensor([18, 47, 56]), the target is: 57
When the input is tensor([18, 47, 56, 57]), the target is: 58
When the input is tensor([18, 47, 56, 57, 58]), the target is: 1
When the input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [ ]:
# Batch dimension (for parallel processing)
# Separate train_data randomly
torch.manual_seed(1105)
batch_size = 4 # num of independent sequences to process in parallel
block_size = 8

def get_batch(split, train_data=train_data, val_data=val_data):
  """Generate a small bach of inputs x and targets y."""
  data = train_data if split == "train" else val_data
  # Generate random numbers of length of (len(data) - block_size)
  # with batch_size num of offsets
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  # y is offset by 1 compared to x
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y
xb, yb = get_batch("train", train_data=train_data, val_data=val_data)
print("Inputs to the transformer: ")
print(xb.shape)
print(xb)
print("Targets: ")
print(yb.shape)
print(yb)

Inputs: 
torch.Size([4, 8])
tensor([[43, 42,  1, 51, 43,  1, 40, 56],
        [ 8,  0, 26, 53, 61,  1, 54, 59],
        [21,  6,  1, 58, 53,  1, 51, 39],
        [58,  1, 40, 47, 58, 58, 43, 56]])
Targets: 
torch.Size([4, 8])
tensor([[42,  1, 51, 43,  1, 40, 56, 53],
        [ 0, 26, 53, 61,  1, 54, 59, 58],
        [ 6,  1, 58, 53,  1, 51, 39, 49],
        [ 1, 40, 47, 58, 58, 43, 56, 50]])


In [ ]:
for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"When the input is {context}, the target is: {target}")

When the input is tensor([24]), the target is: 43
When the input is tensor([24, 43]), the target is: 58
When the input is tensor([24, 43, 58]), the target is: 5
When the input is tensor([24, 43, 58,  5]), the target is: 57
When the input is tensor([24, 43, 58,  5, 57]), the target is: 1
When the input is tensor([24, 43, 58,  5, 57,  1]), the target is: 46
When the input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is: 43
When the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is: 39
When the input is tensor([44]), the target is: 53
When the input is tensor([44, 53]), the target is: 56
When the input is tensor([44, 53, 56]), the target is: 1
When the input is tensor([44, 53, 56,  1]), the target is: 58
When the input is tensor([44, 53, 56,  1, 58]), the target is: 46
When the input is tensor([44, 53, 56,  1, 58, 46]), the target is: 39
When the input is tensor([44, 53, 56,  1, 58, 46, 39]), the target is: 58
When the input is tensor([44, 53, 56,  1, 58, 46, 39, 58]

In [ ]:
torch.manual_seed(1105)
vocab_size = len(chars)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # Each token reads off the logits for next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):
    # idx and targets are both (B, T) tensor of integers
    logits = self.token_embedding_table(idx) # (B, T, C)
    return logits

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)